In [2]:
import cv2
import numpy as np
import random

def augment_image(img):
    h, w, _ = img.shape

    # ---- Rotation ----
    angle = random.randint(-15, 15)
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))

    # ---- Brightness (FIXED) ----
    value = random.randint(-30, 30)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Convert to int to avoid overflow
    v = hsv[:, :, 2].astype(np.int16)
    v = np.clip(v + value, 0, 255)
    hsv[:, :, 2] = v.astype(np.uint8)

    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # ---- Horizontal Flip ----
    if random.random() > 0.5:
        img = cv2.flip(img, 1)

    return img


def double_images(folder_path):
    images = os.listdir(folder_path)
    count = len(images)

    for i, img_name in enumerate(images):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue

        aug_img = augment_image(img)
        new_name = f"aug_{i}.jpg"
        cv2.imwrite(os.path.join(folder_path, new_name), aug_img)

    print(f"Images doubled in: {folder_path}")


# Run for both persons
double_images("dataset/train/tom_cruise")
double_images("dataset/train/will_smith")


Images doubled in: dataset/train/tom_cruise
Images doubled in: dataset/train/will_smith


In [7]:
import os
import cv2
import numpy as np
import random

# Path to Sachin's images
folder = "dataset/train/sachin"

# Step 0: Reduce to 160 images
images = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
num_to_remove = len(images) - 160
if num_to_remove > 0:
    images_to_remove = random.sample(images, num_to_remove)
    for img_name in images_to_remove:
        os.remove(os.path.join(folder, img_name))

# List valid images after removal
images = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

# Augmentation function
def augment_image(img):
    # Random brightness
    value = random.randint(-40, 40)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV).astype(np.int16)
    hsv[:,:,2] = np.clip(hsv[:,:,2] + value, 0, 255)
    hsv = hsv.astype(np.uint8)
    img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # Random horizontal flip
    if random.random() > 0.5:
        img = cv2.flip(img, 1)

    # Random rotation
    angle = random.randint(-25, 25)
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
    img = cv2.warpAffine(img, M, (w, h))

    return img

# Step 1: Augment all remaining 160 images in-place
for img_name in images:
    img_path = os.path.join(folder, img_name)
    img = cv2.imread(img_path)
    if img is None:
        print(f"Skipping unreadable image: {img_name}")
        continue
    aug_img = augment_image(img)
    cv2.imwrite(img_path, aug_img)  # overwrite original

# Final check
images = [f for f in os.listdir(folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
print("Final total images in train folder:", len(images))


Final total images in train folder: 160
